## Treinando uma Rede Neural Convolucional usando o TensorFlow

- Arquitetura simples com duas camadas convolucionais
- Utilizando um conjunto de imagens de células cervicais

## Base de imagens de células cervicais
#### Células anormais
![alt text](abnormal.tif "Abnormal Cell")

#### Células normais
![alt text](normal.tif "Normal Cell")


# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
from skimage.io import imread

# Ler dados

In [ ]:
database_classes = glob('database/*/')
n_classes = len(database_files)
n_files = glob('database/*/*.tif')

labels = np.zeros((len(n_files),n_classes))
images = np.zeros((len(n_files),100,100,3))

cont = 0
lb = 0
for classes in database_classes:
    files_in_classe = glob(classes+'*.tif')
    for fc in files_in_classe:
        labels[cont,lb] = lb
        images[cont,:,:,:] = imread(fc)
        cont+=1
    lb +=1

In [ ]:
database_classes

In [ ]:
plt.imshow(images[4,:,:,:]/255)

# Dividindo dados em Treino e Teste

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.3)

In [ ]:
print('Tamanho do Treino: ', X_train.shape)
print('Tamanho do Teste: ', X_test.shape)

# Construindo a CNN

In [ ]:
import keras
import keras.layers as layers
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from time import time

In [ ]:
model = keras.Sequential()

model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=(100,100,3),name='layer_conv1'))
model.add(layers.MaxPool2D(pool_size=(2, 2),name='layer_pool1'))

model.add(layers.Conv2D(filters=32, kernel_size=(5, 5), activation='relu',name='layer_conv2'))
model.add(layers.MaxPool2D(pool_size=(2, 2),name='layer_pool2'))

model.add(layers.Conv2D(filters=32, kernel_size=(7, 7), activation='relu',name='layer_conv3'))
model.add(layers.MaxPool2D(pool_size=(2, 2),name='layer_pool3'))


model.add(layers.Flatten(name='layer_flatten'))

#model.add(layers.Dropout(rate=0.8,name='layer_drop'))

model.add(layers.Dense(units=256, activation='relu',name='layer_fc1'))

model.add(layers.Dense(units=128, activation='relu',name='layer_fc2'))

model.add(layers.Dense(units=2, activation = 'softmax',name='layer_softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(), metrics=['accuracy'])

In [ ]:

EPOCHS = 10
BATCH_SIZE = 64

In [ ]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size = 0.2)
train_generator = ImageDataGenerator().flow(X_train, Y_train, batch_size=BATCH_SIZE)
validation_generator = ImageDataGenerator().flow(X_validation, Y_validation, batch_size=BATCH_SIZE)

In [ ]:
steps_per_epoch = X_train.shape[0]//BATCH_SIZE
validation_steps = X_validation.shape[0]//BATCH_SIZE

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, 
                    validation_data=validation_generator, validation_steps=validation_steps, 
                    shuffle=True, callbacks=[tensorboard])

In [ ]:
score = model.evaluate(X_test, Y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Classificação com a CNN